### NOTEs
0. eth-tester, just send_transaction.
    but, can customize with other accounts managed by eth-tester
1. once you move over to testnet/mainnet, need to specify where pk is coming from
    you can either, load your account into the script, or create one and fund it within
    WARN: dont commit private keys!
  -- so, build tx, sign it, then send raw tx
2. for contracts: let the contract api build the tx for you, but still need to provide the pk
3. you can use middleware to load and auto-execute txs from a particular account

In [1]:
from web3 import Web3, EthereumTesterProvider

w3 = Web3(EthereumTesterProvider())

print(w3.client_version)
print(w3.api)

EthereumTester/0.8.0b3/darwin/python3.9.7
6.1.0


In [2]:
acct1 = w3.eth.accounts[0]
bal = w3.from_wei(w3.eth.get_balance(acct1), 'ether')
print(f"Account at address {acct1} contains {bal} ether")

Account at address 0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf contains 1000000 ether


In [3]:
some_address = "0x0000000000000000000000000000000000000000"
tx_hash = w3.eth.send_transaction({"to": some_address, "value": 123123123123123})
# ^ that worked, because eth-tester auto-magically used account one as the "from" value

tx = w3.eth.get_transaction(tx_hash)
assert tx["from"] == acct1

In [4]:
tx

AttributeDict({'type': 2,
 'hash': HexBytes('0x07753e07132115df2daf28e2103c955335017b5761647b59e2bee9d6078fd1df'),
 'nonce': 0,
 'blockHash': HexBytes('0x71a9ff4087c1f2bf88dc50f2e3a4018a32cfcf931007340965afc4afa0a0f2f1'),
 'blockNumber': 1,
 'transactionIndex': 0,
 'from': '0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf',
 'to': '0x0000000000000000000000000000000000000000',
 'value': 123123123123123,
 'gas': 121000,
 'data': '0x',
 'r': HexBytes('0x1e35b554dd763a39135756602bb8ac64e4219b32fba4831b9828c318dbe40e09'),
 's': HexBytes('0x7c65b5018881f656da8738a5fd26b74241227837a0a7b2a825b35e35763a888e'),
 'v': 0,
 'chainId': 131277322940537,
 'maxFeePerGas': 1000000000,
 'maxPriorityFeePerGas': 1000000000,
 'accessList': [],
 'gasPrice': 1000000000})

this dont work in testnets or mainnet!

at least not as is. you can use middleware.

In [5]:
# middleware option:
# from web3 import Web3, HTTPProvider
from web3.middleware import construct_sign_and_send_raw_middleware
from eth_account import Account
import os

# example private key. never commit your key in your code! grab from env var instead:
# pk = os.environ.get('PRIVATE_KEY')
pk = '0x7ec01898a720beda9304921aec62735846aa1a545a8332bc030577c21f72cd46'

acct2 = w3.eth.account.from_key(pk)
# fund the new account, so it has something to send in a transaction:
w3.eth.send_transaction({"from": acct1, "value": 9999999999999999, "to": acct2.address})
print(f"acct2 balance: {w3.eth.get_balance(acct2.address)}")

# add acct2 as auto signer
w3.middleware_onion.add(construct_sign_and_send_raw_middleware(acct2))

# optionally can make it the default signer as well
w3.eth.default_account = acct2.address

tx_hash = w3.eth.send_transaction({"value": 3333333333, "to": some_address})
tx = w3.eth.get_transaction(tx_hash)
assert tx["from"] == acct2.address

acct2 balance: 9999999999999999


if you don't opt for the middleware, you'll need to: 
- build each transaction, 
- sign it, and 
- then use send_raw_transaction.

In [6]:
# 1. build a new tx
transaction = {
    'from': acct2.address,
    'to': some_address,
    'value': 1000000000,
    'nonce': w3.eth.get_transaction_count(acct2.address),
    'gas': 200000,
    'type': '0x2',
    'maxFeePerGas': 2000000000,
    'maxPriorityFeePerGas': 1000000000,
}

# 2. sign tx with a private key
signed = w3.eth.account.sign_transaction(transaction, pk)

# 2.a. the raw transaction is what will be broadcast to the network:
print(f"Raw tx: {signed.rawTransaction}")

# 3. send the signed transaction
tx_hash = w3.eth.send_raw_transaction(signed.rawTransaction)
tx = w3.eth.get_transaction(tx_hash)
assert tx["from"] == acct2.address

Raw tx: b'\x02\xf8o\x80\x01\x84;\x9a\xca\x00\x84w5\x94\x00\x83\x03\r@\x94\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x84;\x9a\xca\x00\x80\xc0\x80\xa0\x07\xe2\xf4\xddd\xa5jl~\xfc\x1b\\\x8cG\xbf~,\xf3W%H\xdb\xe5\xc1>\x13\xe6\xf4L\x05\x90\xcf\xa0Xt\x0b\x15\x994\xdb"\xfd\x84\xa6\x83\xed\xfdLZi\x02%g\xd4<\x9a\x184z\xd4\xab\xcdWmP'


In [7]:
# last up, build, sign, send contract txs:

# TODO: get bytecode for a smol dummy contract to deploy right quick
unicorns = w3.eth.contract(address="0xfB6916095ca1df60bB79Ce92cE3Ea74c37c5d359", abi=EIP20_ABI)

# Build a transaction that invokes this contract's function, called transfer
unicorn_txn = unicorns.functions.transfer(
    '0xfB6916095ca1df60bB79Ce92cE3Ea74c37c5d359',
    1,
).build_transaction({
    'chainId': 1,
    'gas': 70000,
    'maxFeePerGas': w3.to_wei('2', 'gwei'),
    'maxPriorityFeePerGas': w3.to_wei('1', 'gwei'),
    'nonce': w3.eth.get_transaction_count(acct2.address),
})

print(f"tx params: {unicorn_txn}")

private_key = b"\xb2\\}\xb3\x1f\xee\xd9\x12''\xbf\t9\xdcv\x9a\x96VK-\xe4\xc4rm\x03[6\xec\xf1\xe5\xb3d"
signed_txn = w3.eth.account.sign_transaction(unicorn_txn, private_key=private_key)

signed_txn.hash
HexBytes('0x748db062639a45e519dba934fce09c367c92043867409160c9989673439dc817')

print(f"Raw tx: {signed_txn.rawTransaction}")

tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)  
tx = w3.eth.get_transaction(tx_hash)
assert tx["from"] == acct2.address

# When you run send_raw_transaction, you get the same result as the hash of the transaction:
w3.to_hex(w3.keccak(signed_txn.rawTransaction))
'0x748db062639a45e519dba934fce09c367c92043867409160c9989673439dc817'

NameError: name 'EIP20_ABI' is not defined